# Lesson 5 project

## Setup

### This Demo Requires an OpenAI “Secret Key”

Unlike the APIs in the previous demo, OpenAI requires users of its APIs to register for an account and use that account to create one or more API keys — which they call _secret keys_ — in order to use the API. The API is _not_ free of charge to use, which means that you need a credit card, which you use to add money to your OpenAI account balance.

Fortunately, it doesn’t cost very much money to use the OpenAI API when you’re using it on personal projects. At the time of writing, US$5.00 should be enough to cover thousands of input and output messages to and from the API.

OpenAI, like many other companies that provide generative AI APIs, likes to change the user interface on their dashboards often. Because of this, we have to leave it to you to perform the following steps on your own:

1. Sign up for an OpenAI account.
2. While logged into the OpenAI dashboard under your account, create a secret key.

Once you have created a secret key, copy that key. Create a new text file named `.env` containing the following:

```
OPENAI_API_KEY = replace_this_with_your_secret_key
```

Replace `replace_this_with_your_secret_key` with your secret key and save the file to the directory where you will create the Jupyter notebook for this demo. By storing the API key in a .env file (and making sure that the .env is _not_ added to source control), you avoid “hard-coding” the API key into your application, which makes it more likely that someone unauthorized will discover it.

### Install the Necessary Packages

In [ ]:
! pip install dotenv
# If the above dotenv installation doesn’t work, try
#! pip install python-dotenv

In [ ]:
! pip install openai
# If the above openai installation doesn’t work, try
# pip3 install openai

These `pip` commands will install the following libraries:

- `dotenv`, which provides functions for reading `.env` files (such as the one you just created) to create environment variables. 
- `openai`, which provides functionality for communicating with OpenAI’s AI models.

## Creating an OpenAI Client Object

The first step in building an OpenAI-powered application is to create an OpenAI client object. This object enables access to OpenAI’s various APIs, including the two we’ll use: GPT and DALL-E.

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

# When called without arguments, OpenAI will try to get
# the API key from the OPENAI_API_KEY environment variable.
load_dotenv()
client = OpenAI()

The `load_dotenv()` method, provided by the `dotenv` library, reads the contents of the `.env` file to create the `OPENAI_API_KEY` environment variable. This variable isn’t a variable in your application, but in your operating system, the environent where your application runs (hence the name).

The call to `OpenAI()`, which creates an OpenAI client object, looks for an environment variable named `OPENAI_API_KEY`. If it finds this environment variable, and if its value corresponds to a legitimate API key from an OpenAI account with money it its balance, the client object is instantiated and can be used to access OpenAI’s APIs.

## Creating Chat Completions

When you use ChatGPT, you are providing an OpenAI LLM with prompts and it responds with _chat completions_. To access OpenAI’s LLMs, you use the client object’s `chat` property, whose properties provide access to its chat completion API.

### Defining a Chat Completion Function

Let’s create a function that generates a “single-turn” completion, where an OpenAI GPT model is given a single prompt and generates a response or solution in a single step, without any follow-up. 

In [ ]:
def create_completion(
    user_prompt, 
    model="gpt-4o-mini", 
    system_prompt="""
    You are a highly knowledgeable, conversational, and polite assistant capable of
    discussing a wide range of topics, from casual conversations to complex technical subjects.
    Your goal is to provide accurate, concise, and clear answers while maintaining
    a friendly and approachable tone.
    """, 
    temperature=1.0
):
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
    )
    return completion.choices[0].message.content

This function, `create_completion()`, simplifies making a call to the OpenAI client’s `chat.completions.create()` method, which creates chat completions based on a list of messages that you pass to it.

In our call to `chat.completions.create()`, we provide arguments for the following parameters:

<table>
    <tr>
        <td><strong>Parameter</strong></td>
        <td style="text-align:left;"><strong>Description</strong></td>
    </tr>
    <tr>
        <td><code>model</code></td>
        <td>
            <p>A string value that specifies which LLM should provide the completion. In this demo, we’ll use `gpt-4o-mini`, the smallest, quickest, and cheapest of the current models, and more than enough for our application.</p>
        </td>
    </tr>
    <tr>
        <td><code>temperature</code></td>
        <td>
            <p>A float value that controls the level of randomness in the completion text that the model generates. Temperatures closer to 0 result in completions that are generated more quickly, and are more deterministic and predictable. At the default value of 1, the completions are supposed to responses with a mix of creativity and consistency. You can provide temperatures higher than 1 for even more creative results (that take longer to generate), but at around 1.4 and higher, the LLM tends to produce nonsensical results.</p>
        </td>
    </tr>
    <tr>
        <td><code>messages</code></td>
        <td style="text-align:left;">
            <p>A list of dictionaries, where each dictionary represents a message to the LLM. Since this demo in concerned only with a single-turn completion, the `messages` list will contain only two message dictionaries:</p>
            <ol>
                <li>A message with instructions that define how the LLM should behave or responds. This is called a <em>system message</em>, and it’s specified as such by setting the value of the <code>role</code> key to <code>system</code>. The actual message is defined as the value for the <code>content</code> key.
                </li>
                <li>A message containing the user’s prompt. This is called a _user message_, which is designated by setting the value of the `role` key to `user`. The user’s prompt is used as the value for the <code>content</code> key.</li>
            </ol>
        </td>
    </tr>
</table>

`chat.completions.create()` returns a completion object containing all sorts of information that’s beyond the scope of this demo. The part of the object we’re most interested in is its `choices[0].message.content` property, which contains the completion generated by the LLM. This is what our `create_completion()` function returns.

## Create a chat completion function

`chat.completions.create()` returns a completion object containing all sorts of information that’s beyond the scope of this demo. The part of the object we’re most interested in is its `choices[0].message.content` property, which contains the completion generated by the LLM. This is what our `create_completion()` function returns.

### Using the Chat Completion Function

Test the newly-defined `create_completion()`:

In [ ]:
create_completion("I need a character for a pulp novel about a trucker who moonlights as a private detective and gets into wacky but plausible adventures.")

See how creative the LLM can get by boosting the temperature to `1.25`:

In [ ]:
create_completion("I need a character for a pulp novel about a trucker who moonlights as a private detective and gets into wacky but plausible adventures.", temperature=1.25)

And finally, observe what happens when you _really_ heat things up and bring up the temperature to `1.6`:

In [ ]:
create_completion("I need a character for a pulp novel about a trucker who moonlights as a private detective and gets into wacky but plausible adventures.", temperature=1.6)

The other thing you can experiment with is `create_completion()`’s `system_prompt` parameter. Try overriding the default system message by instructing the LLM to behave very differently. 

In [ ]:
create_completion("Explain the second law of thermodynamics.", system_prompt="You are the stereotypical used car salesperson from a late 1970s low-budget comedy.", temperature=1.2)

## Creating AI-Generated Images

OpenAI’s text-to-image AI model DALL-E debuted in January 2021. It preceded ChatGPT by nearly two years, and its API release in November 2022 was timed to coincide the launch of ChatGPT (whose API would not be released to the public until march 2023). To access the DALL-E API, you use the client object’s `images` property.

### Defining an Image Generation Function

Just as we wrote a function to simplify the function call to OpenAI for a chat completion, let’s write a function to simply the OpenAI function call for generating an image.

In [ ]:
def create_image(
    user_prompt,
    size="1024x1024",
    quality="standard"
):
    response = client.images.generate(
        model="dall-e-3",
        prompt=user_prompt,
        size=size,
        quality=quality
    )
    return response.data[0].url

This function, `create_image()`, simplifies making a call to the OpenAI client’s `images.generate()` method, which creates images based on the prompt that you pass to it.

In our call to `images.generate()`, we provide arguments for the following parameters:

<table>
    <tr>
        <td><strong>Parameter</strong></td>
        <td style="text-align:left;"><strong>Description</strong></td>
    </tr>
    <tr>
        <td><code>model</code></td>
        <td>
            <p>A string value that specifies which model should generate the completion. In this demo, we’ll use <code>dall-e-3</code>. This is not the default model — that’s <code>dall-e-2</code> — but this one generates more satisfying images.</p>
        </td>
    </tr>
    <tr>
        <td><code>prompt</code></td>
        <td>
            <p>A string value containing a description of the image to be generated.</p>
        </td>
    </tr>
    <tr>
        <td><code>size</code></td>
        <td>
            <p>A string specifying one of a small set of image sizes. We’re using the default <code>1024x1024</code> value in this demo; you can see the full set of sizes and the models for which they are applicable in <a href="https://platform.openai.com/docs/api-reference/images/create">OpenAI’s images API documentation</a>.</p>
        </td>
    </tr>
    <tr>
        <td><code>quality</code></td>
        <td>
            <p>A string determining the quality of the image to be generated. We’re using the default value of <code>standard</code> for this demo, but for even higher quality images, use the value <code>hd</code>. This parameter works only for the <code>dall-e-3</code> model.</p>
        </td>
    </tr>
</table>

### Using the Image Generation Function

Test the newly-defined `create_image()`:

In [ ]:
create_image("A shiba inu in a cowboy hat in a retro-western-futuristic rodeo riding a Sputnik satellite that's bucking like a bronco.")

The result will be a URL for the generated image. The URL will be valid for an hour.

## Building a “Smart” Weather Application

Let’s take what we’ve learned so far in this demo and the previous demo and build an application that:

- Takes the name of a location
- Gets the current weather at that location
- Delivers the weather report for that location in the form of a poem that also tells you if you should wear a sweater or bring an umbrella
- And finally, draws a picture of the weather at that location

### Converting the Name of a Place to Latitude and Longitude

The Open-Meteo API is the natural choice for getting the current weather for a given location. The problem is that it takes latitude and longitude as its location arguments, not place names. We’ll need some way to convert a location name into coordinates.

Fortunately, the [GeoPy](https://geopy.readthedocs.io/en/stable/) has what we need: [Nominatim](https://nominatim.org/), a geocoder, which uses [OpenStreetMap](https://www.openstreetmap.org/) data to convert addresses or place names to latitude and longitude.

First, you’ll have to install `GeoPy`.

In [ ]:
! pip install geopy
# If the above dotenv installation doesn’t work, try
#! pip install python-dotenv

Define a function that uses `Nominatim` to convert an address or place name into its latitude and longitude:

In [ ]:
from geopy.geocoders import Nominatim

def location_name_to_latlong(location_name):
    geolocator = Nominatim(user_agent="Kodeco - Python for AI demo")
    location = geolocator.geocode(location_name)
    return (location.latitude, location.longitude)

The code creates `geolocator`, a `Nominatim` geolocator object. The `user_agent` parameter should contain the name of the app using the `Nominatim` service. We use `Nominatim`’s `geocode()` method to get the coordinates for the given location and return those coordinates as a tuple.

Try it out by using it to get the latitude and longitude of the original Radio Shack store:

In [ ]:
# What are the coordinates for the original Radio Shack store?
location_name_to_latlong("46 Brattle St., Cambridge, MA, USA")

It’s not limited to addresses — if a place is listed in OpenStreetMap, our function can convert its name into coordinates:

In [ ]:
# What are the coordinates for the roadside attraction known
# as “Carhenge” — like Stonehenge, but made of cars?
location_name_to_latlong("Carhenge")

### Getting the Current Weather at a Location

Let’s write a couple of functions to get the weather for a given location. This will use the `location_name_to_latlong()` function we just defined and the Open-Meteo API.

In [ ]:
import requests
import json

def celsius_to_fahrenheit(degrees_celsius):
    return (degrees_celsius * 1.8) + 32

def get_current_weather(location_name):
    WEATHER_CODE_TABLE = {
        0: "clear sky",
        1: "mainly clear",
        2: "partly cloudy",
        3: "overcast",
        45: "fog",
        48: "depositing rime fog",
        51: "light drizzle",
        53: "moderate drizzle",
        55: "dense drizzle",
        56: "light freezing drizzle",
        57: "dense freezing drizzle",
        61: "slight rain",
        63: "moderate rain",
        65: "heavy rain",
        66: "light freezing rain",
        67: "heavy freezing rain",
        71: "slight snow",
        73: "moderate snow",
        75: "heavy snow",
        77: "snow grains",
        80: "light rain showers",
        81: "moderate rain showers",
        82: "violent rain showers",
        85: "slight snow showers",
        86: "heavy snow showers",
        95: "thunderstorm",
        96: "thunderstorm with slight hail",
        99: "thunderstorm with heavy hail",
    }
    
    latitude, longitude = location_name_to_latlong(location_name)
    url = "https://api.open-meteo.com/v1/forecast/"
    query_parameters = {
        "latitude":  latitude,
        "longitude": longitude,
        "current":   "weathercode,temperature_2m,cloudcover,relativehumidity_2m"
    }
    response = requests.get(url, params=query_parameters)
    data = response.json()
    result = {
        "weather": WEATHER_CODE_TABLE.get(data["current"]["weathercode"], "unknown"),
        "cloud_cover": data["current"]["cloudcover"],
        "temperature": celsius_to_fahrenheit(data["current"]["temperature_2m"]),
        "humidity": data["current"]["relativehumidity_2m"],
    }
    return result

The `get_current_weather()` method takes a location name, uses `location_name_to_latlong()` to convert it into coordinates, and then uses them as two of the parameters for the GET request it sends to Open-Meteo. The final parameter, `current`, contains the weather information we want in the response:

<table>
    <tr>
        <td><strong>Parameter string</strong></td>
        <td style="text-align:left;"><strong>Description</strong></td>
    </tr>
    <tr>
        <td><code>weathercode</code></td>
        <td>
            <p>An integer representing the current weather at the location. We’ll use the <code>WEATHER_CODE_TABLE</code> dictionary to convert it into words.</p>
        </td>
    </tr>
    <tr>
        <td><code>temperature_2m</code></td>
        <td>
            <p>A float representing the current temperature at the location at an altitude of 2 meters (about 6 feet) above ground level. This value is in degrees Celsius, but we’ve also provide a function to convert Celsius to Fahrenheit.</p>
        </td>
    </tr>
    <tr>
        <td><code>cloudcover</code></td>
        <td>
            <p>The amount of cloud cover at the location, expressed as a percentage between 0 and 100 inclusive.</p>
        </td>
    </tr>
    <tr>
        <td><code>relativehumidity_2m</code></td>
        <td>
            <p>The relative humidity at the requested location, as measured at a height of 2 meters (about 6 feet) above the ground. This value is expressed as a percentage — between 0 and 100 inclusive.</p>
        </td>
    </tr>
</table>

The function returns a dictionary containing the values for the weather properties listed above.

Test the method by getting the weather at your current location. For me, that’s Tampa, Florida in the United States:

In [ ]:
get_current_weather("Tampa, Florida, USA")

### Generating a Weather Poem

We now have everything we need to create the two functions for our smart weather application. Let’s create a function to generate the weather poem.

In [ ]:
def create_weather_poem(location):
    weather = get_current_weather(location)
    weather_prompt = (
        f"The current weather in {location} is {weather['weather']}, " +
        f"with a temperature of {weather['temperature']} degrees F, " +
        f"{weather['cloud_cover']}% cloud cover, and " +
        f"{weather['humidity']}% humidity. " +
        "Create a poem about this location and its current weather. " +
        "The poem should also suggest if someone would need a sweater " +
        "or umbrella for the current weather."
    )
    return create_completion(weather_prompt)

`create_weather_poem()` uses the results from `get_current_weather()` to assemble a prompt, which it feeds to `create_completion()` to generate a weather poem.

Try it out by generating a weather poem for where you are right now — or any other location where you want to know the weather.

In [ ]:
create_weather_poem("Tampa, Florida, USA")

### Generating a Weather Image

Generating a weather image follows a process similar to that for generating the weather poem: retrieve the weather information for the given location, use that information to build a prompt, and pass that prompt to the appropriate API.

In [ ]:
def create_weather_image(location):
    weather = get_current_weather(location)
    weather_prompt = (
        f"A skyline view of {location}, where the weather is {weather['weather']}, " +
        f"with a temperature of {weather['temperature']} degrees F, " +
        f"{weather['cloud_cover']}% cloud cover, and " +
        f"{weather['humidity']}% humidity. " +
        "Include people dressed appropriately for the weather."
    )
    weather_image_url = create_image(weather_prompt)
    return weather_image_url

Once again, test the function:

In [ ]:
create_weather_image("Tampa, Florida, USA")

### Bringing It All Together

We’ve done a lot in this demo, so let’s make the application a simple one that asks the user for location and prints out the poem and picture URL. You may find yourself doing it over and over, just to see the what comes out:

In [ ]:
print("Sweater Weather, the poetic weather app")
print("=======================================")
location = input("What place would you like a weather poem and picture for?")
weather_poem = create_weather_poem(location)
weather_image_url = create_weather_image(location)

print(f"A weather poem about {location}:")
print(f"{weather_poem}\n")
print("And here's a picture (click to view it):")
print(weather_image_url)